In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import pi
from statsmodels.stats.power import TTestIndPower
from scipy import stats

from bokeh.plotting import figure, show, curdoc
from bokeh.io import output_file
from bokeh.layouts import row, column, gridplot
from bokeh.models import ColumnDataSource, Legend, BoxAnnotation, HoverTool, Plot, FactorRange, TapTool, OpenURL, Range1d
from bokeh.models.widgets import Tabs, Panel

import statsmodels.api as sm

In [2]:
!wget "https://docs.google.com/spreadsheets/d/1O8YItuJ4CtaFFEXbjBcxvhUoM7v9AX1uFDRMhh7iQZM/export?format=xlsx" -O "../metadata.xlsx"

--2023-01-25 16:30:51--  https://docs.google.com/spreadsheets/d/1O8YItuJ4CtaFFEXbjBcxvhUoM7v9AX1uFDRMhh7iQZM/export?format=xlsx
Resolving docs.google.com (docs.google.com)... 142.250.138.102, 142.250.138.101, 142.250.138.113, ...
Connecting to docs.google.com (docs.google.com)|142.250.138.102|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: https://doc-14-5g-sheets.googleusercontent.com/export/mq6he3r7ig44qobar1fsg51390/66shevbaer6bovsdsef3velagk/1674685850000/102025844088317935117/*/1O8YItuJ4CtaFFEXbjBcxvhUoM7v9AX1uFDRMhh7iQZM?format=xlsx [following]
--2023-01-25 16:30:51--  https://doc-14-5g-sheets.googleusercontent.com/export/mq6he3r7ig44qobar1fsg51390/66shevbaer6bovsdsef3velagk/1674685850000/102025844088317935117/*/1O8YItuJ4CtaFFEXbjBcxvhUoM7v9AX1uFDRMhh7iQZM?format=xlsx
Resolving doc-14-5g-sheets.googleusercontent.com (doc-14-5g-sheets.googleusercontent.com)... 142.251.32.193
Connecting to doc-14-5g-sheets.googleusercontent.com (doc-14-5g

In [3]:
data = pd.read_excel('../metadata.xlsx', sheet_name='WearableBP Paper Metadata')

# Housekeeping

In [4]:
import nltk
from rake_nltk import Rake
nltk.download('stopwords')

def find_keywords(x, include_columns):
    s = ''
    for col in include_columns:
        if x[col] != x[col]:
            pass
        else:
            s += ' ' + str(x[col])
            
    rake_nltk_var = Rake()
    rake_nltk_var.extract_keywords_from_text(s)
    keywords = rake_nltk_var.get_ranked_phrases()
    return ' '.join(keywords)

include_columns = ['Authors', 'Title', 'Source', 'Exclude Reason', 'Key Devices and Measurements', 'Calibration Technique', 'Algorithm', 'Dataset', 'Training Subject Characteristics', 'Testing Subject Characteristics', 'Study Characteristics','Evaluation Metric', 'Notes', 'Abstract']
data['Keywords'] = data.apply(find_keywords, axis=1, args=(include_columns, ))

[nltk_data] Downloading package stopwords to /Users/matt/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
a = data.copy().dropna(subset='Exclude Reason')
len(a[a['Exclude Reason'].str.contains('record split without personalization')])

330

In [6]:
def compress_authors(x):
    a = x.split(',')[0].split(' ')
    if len(a) == 1:
        return a[0]
    elif len(a) == 2:
        return a[1]
    elif len(a) == 3:
        return a[2]
    else:
        return a[-1]

df = data.copy()
df = df[(df['Include'] == 1) & (df['BP Distribution Type'] == 'ME') & ((df['Evaluation Metric'] == 'MAE') | (df['Evaluation Metric'] == 'ME'))]

df['Sensor Data'] = df['Key Devices and Measurements'].str.split(';', expand=True)[0]
df['Calibration Technique'] = df['Calibration Technique'].str.split(';', expand=True)[0]
df['Algorithm'] = df['Algorithm'].str.split(';', expand=True)[0]
df['Dataset'] = df['Dataset'].str.split(';', expand=True)[0]
df['Testing Subject Characteristics'] = df['Testing Subject Characteristics'].str.split(';', expand=True)[0]
df['Study Characteristics'] = df['Study Characteristics'].str.split(';', expand=True)[0]
df['Algorithm'] = df['Algorithm'].str.split(';', expand=True)[0]
df['SBP Distribution Raw'] = df['BP Distribution'].str.split(';', expand=True)[0]
df['SBP Distribution STD'] = df['SBP Distribution Raw'].str.split('±', expand=True)[1].astype(float)
df['DBP Distribution Raw'] = df['BP Distribution'].str.split(';', expand=True)[1]
df['DBP Distribution STD'] = df['DBP Distribution Raw'].str.split('±', expand=True)[1].astype(float)
df['SBP Reported Error Raw'] = df['Reported Result'].str.split(';', expand=True)[0]
df['DBP Reported Error Raw'] = df['Reported Result'].str.split(';', expand=True)[1]

In [7]:
df_mae = df[df['Evaluation Metric'] == 'MAE']
df_mae['SBP Error STD'] = df_mae['SBP Reported Error Raw'].astype(float)*np.sqrt(pi/2)
df_mae['DBP Error STD'] = df_mae['DBP Reported Error Raw'].astype(float)*np.sqrt(pi/2)
df_me = df[df['Evaluation Metric'] == 'ME']
df_me['SBP Error STD'] = df_me['SBP Reported Error Raw'].str.split('±', expand=True)[1].astype(float)
df_me['DBP Error STD'] = df_me['DBP Reported Error Raw'].str.split('±', expand=True)[1].astype(float)
df_concat = pd.concat([df_me, df_mae], axis=0)
df_concat['SBP ED'] = df_concat['SBP Distribution STD']/df_concat['SBP Error STD']
df_concat['DBP ED'] = df_concat['DBP Distribution STD']/df_concat['DBP Error STD']

df_concat['SBP ED 95CI UB'] = df_concat.apply(lambda x: np.sqrt(stats.f.isf(0.05/2, x['Number of Test Subjects']-1, x['Number of Test Subjects']-1))*x['SBP ED'], axis=1)
df_concat['SBP ED 95CI LB'] = df_concat.apply(lambda x: np.sqrt(stats.f.isf(1-0.05/2, x['Number of Test Subjects']-1, x['Number of Test Subjects']-1))*x['SBP ED'], axis=1)
df_concat['SBP ED MOE'] = (df_concat['SBP ED 95CI UB'] - df_concat['SBP ED 95CI LB'])/2
df_concat['DBP ED 95CI UB'] = df_concat.apply(lambda x: np.sqrt(stats.f.isf(0.05/2, x['Number of Test Subjects']-1, x['Number of Test Subjects']-1))*x['DBP ED'], axis=1)
df_concat['DBP ED 95CI LB'] = df_concat.apply(lambda x: np.sqrt(stats.f.isf(1-0.05/2, x['Number of Test Subjects']-1, x['Number of Test Subjects']-1))*x['DBP ED'], axis=1)
df_concat['DBP ED MOE'] = (df_concat['DBP ED 95CI UB'] - df_concat['DBP ED 95CI LB'])/2

df_concat['Authors'] = df_concat['Authors'].apply(compress_authors) + ' et al.'
compute_power = lambda x: TTestIndPower().solve_power(5/8, ratio=1, alpha=0.05, nobs1=x, power=None, alternative='two-sided')
df_concat['Power'] = df_concat['Number of Test Subjects'].apply(compute_power)
# 2*((stats.norm.isf(1-0.05/2) + stats.norm.isf(1-0.02))/(5/8))**2
# 1-stats.norm.sf(np.sqrt(85/2)*(5/8)-stats.norm.ppf(1-0.05/2))

/var/folders/4k/jr4_f2px17b2sdv74jmvp4x80000gn/T/ipykernel_25023/4128608041.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mae['SBP Error STD'] = df_mae['SBP Reported Error Raw'].astype(float)*np.sqrt(pi/2)
/var/folders/4k/jr4_f2px17b2sdv74jmvp4x80000gn/T/ipykernel_25023/4128608041.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mae['DBP Error STD'] = df_mae['DBP Reported Error Raw'].astype(float)*np.sqrt(pi/2)
/var/folders/4k/jr4_f2px17b2sdv74jmvp4x80000gn/T/ipykernel_25023/4128608041.py:5: 

# Create Subject Split and Personalization Tables

In [8]:
save_cols = ['Title', 'ArticleURL', 'Authors', 'Year', 'Calibration Technique', 'Sensor Data', 'Algorithm', 'Dataset', 'Number of Test Subjects', 'Testing Subject Characteristics', 'Study Characteristics', 'SBP Distribution STD', 'DBP Distribution STD', 'Reported Result', 'SBP ED', 'DBP ED', 'SBP ED MOE', 'DBP ED MOE', 'Power', 'Original Model Implementation Availability', 'Original Model Implementation Link', 'Exact Dataset/Pre-processing Code Availability', 'Exact Dataset/Pre-processing Code Link']

def make_clickable(name, url):
    return '<a href="' + url + '">' + name + '</a>'

save_table = df_concat.copy()[save_cols]
save_table['Title'] = make_clickable(save_table['Title'] + ' (' + save_table['Authors'] + ', '+ save_table['Year'].astype(int).astype(str) + ')', save_table['ArticleURL'])
save_table.drop(['Year'], axis=1, inplace=True)
save_table.drop(['Authors'], axis=1, inplace=True)
save_table.drop(['ArticleURL'], axis=1, inplace=True)

save_table['Original Model Implementation Availability'] = make_clickable(save_table['Original Model Implementation Availability'], save_table['Original Model Implementation Link'])
save_table['Exact Dataset/Pre-processing Code Availability'] = make_clickable(save_table['Exact Dataset/Pre-processing Code Availability'], save_table['Exact Dataset/Pre-processing Code Link'])
save_table.drop(['Original Model Implementation Link', 'Exact Dataset/Pre-processing Code Link'], axis=1, inplace=True)

sls_save_table = save_table[save_table['Calibration Technique'] == 'subject split']
per_save_table = save_table[save_table['Calibration Technique'] == 'personalization']

save_table.to_html('./_includes/include_table.html', index=False, render_links=True, table_id='itable', escape=False)
sls_save_table.to_html('./_includes/include_sls_table.html', index=False, render_links=True, table_id='islstable', escape=False)
per_save_table.to_html('./_includes/include_per_table.html', index=False, render_links=True, table_id='ipertable', escape=False)

# Create Interactive Metadata Statistics Figure

In [9]:
cat_columns = ['Sensor Data', 'Algorithm', 'Dataset', 'Testing Subject Characteristics', 'Study Characteristics']
cont_columns = ['Number of Test Subjects', 'SBP Distribution STD', 'DBP Distribution STD', 'SBP ED', 'DBP ED', 'Power']

tooltips = [("% of subject split", "@{subject split pct}"), 
            ("% of personalization", "@{personalization pct}")]

ps = []
for c in cat_columns:
    d = pd.DataFrame(save_table[[c, 'Calibration Technique']].value_counts()).reset_index().rename({0: 'Count'}, axis=1).pivot(index=c, columns='Calibration Technique', values='Count').reset_index().fillna(0)
    d['subject split pct'] = d['subject split']/d['subject split'].sum()*100
    d['personalization pct'] = d['personalization']/d['personalization'].sum()*100
    d['total'] = d['subject split']+d['personalization']
    d = d.sort_values(by='total')
    source = ColumnDataSource(d)
    p = Plot(title=None, width=800, height=600, toolbar_location=None)
    p = figure(y_range=FactorRange(factors=d[c].unique()), plot_width=1000, plot_height=400, title='',
                toolbar_location=None, tooltips=tooltips, tools = [])
    p.hbar_stack(['subject split', 'personalization'], y=c, source=source, fill_color=['red', 'blue'])
    p.x_range = Range1d(0, (d['subject split']+d['personalization']).max())
    p.yaxis.axis_label = c
    p.xaxis.axis_label = 'Number of Studies'
    ps.append(p)
    output_file('./_includes/' + c.replace(' ', '_') + '_stats.html')
    show(p)

for c in cont_columns:
    d = save_table[[c, 'Calibration Technique']]
    d1 = d[d['Calibration Technique'] == 'subject split'].dropna()
    d2 = d[d['Calibration Technique'] == 'personalization'].dropna()
    p1 = figure(title='Subject Split', width=500, height=400, min_border=0, tools='', background_fill_color="#fafafa", toolbar_location='above')
    p2 = figure(title='Personalization', width=500, height=400, min_border=0, tools=[], background_fill_color="#fafafa", toolbar_location='above')
    hist1, edge1 = np.histogram(d1[c], bins=10)
    hist2, edge2 = np.histogram(d2[c], bins=10)
    p1.quad(top=hist1, bottom=0, fill_color='red', line_color="white", left=edge1[:-1], right=edge1[1:])
    p2.quad(top=hist2, bottom=0, fill_color='blue', line_color="white", left=edge2[:-1], right=edge2[1:])
    
    p1.y_range = Range1d(0, hist1.max())
    p1.xaxis.axis_label = c
    p1.yaxis.axis_label = 'Number of Studies'
    p2.y_range = Range1d(0, hist2.max())
    p2.xaxis.axis_label = c
    p2.yaxis.axis_label = 'Number of Studies'
    p = row(p1, p2)
    ps.append(p)
    output_file('./_includes/' + c.replace(' ', '_') + '_stats.html')
    show(p)
    

# tabs = []
# for i in range(len(cat_columns)):
#     tabs.append(Panel(child=ps[i], title=cat_columns[i]))
# for i in range(len(cont_columns)):
#     tabs.append(Panel(child=ps[len(cat_columns)+i], title=cont_columns[i]))
# figure_tabs = Tabs(tabs=tabs)
# output_file('./_includes/metadata_stats.html')
# show(figure_tabs)

# Create Metadata Statistics for Paper Figure

In [10]:
cat_columns = ['Sensor Data', 'Algorithm', 'Dataset', 'Study Characteristics']

tooltips = [("% of subject split", "@{subject split pct}"), 
            ("% of personalization", "@{personalization pct}")]

ps = []
for c in cat_columns:
    d = pd.DataFrame(save_table[[c, 'Calibration Technique']].value_counts()).reset_index().rename({0: 'Count'}, axis=1).pivot(index=c, columns='Calibration Technique', values='Count').reset_index().fillna(0)
    d['subject split pct'] = d['subject split']/d['subject split'].sum()*100
    d['personalization pct'] = d['personalization']/d['personalization'].sum()*100
    source = ColumnDataSource(d)
    p = Plot(title=None, width=800, height=600, toolbar_location=None)
    p = figure(y_range=FactorRange(factors=d[c].unique()), plot_width=1000, plot_height=400, title='',
                toolbar_location=None, tooltips=tooltips, tools = [])
    p.hbar_stack(['subject split', 'personalization'], y=c, source=source, fill_color=['red', 'blue'])
    p.x_range = Range1d(0, (d['subject split']+d['personalization']).max())
    p.xaxis.axis_label = c
    p.yaxis.axis_label = 'Count'
    ps.append(p)

for c in cont_columns:
    d = save_table[[c, 'Calibration Technique']]
    d1 = d[d['Calibration Technique'] == 'subject split'].dropna()
    d2 = d[d['Calibration Technique'] == 'personalization'].dropna()
    p1 = figure(title=c + ' (Subject Split)', width=500, height=400, min_border=0, tools='', background_fill_color="#fafafa", toolbar_location='above')
    p2 = figure(title=c + ' (Personalization)', width=500, height=400, min_border=0, tools=[], background_fill_color="#fafafa", toolbar_location='above')
    hist1, edge1 = np.histogram(d1[c], bins=10)
    hist2, edge2 = np.histogram(d2[c], bins=10)
    p1.quad(top=hist1, bottom=0, fill_color='red', line_color="white", alpha=0.5, left=edge1[:-1], right=edge1[1:])
    p2.quad(top=hist2, bottom=0, fill_color='blue', line_color="white", alpha=0.5, left=edge2[:-1], right=edge2[1:])
    
    p1.y_range = Range1d(0, hist1.max())
    p1.xaxis.axis_label = c
    p1.yaxis.axis_label = 'Count'
    p2.y_range = Range1d(0, hist2.max())
    p2.xaxis.axis_label = c
    p2.yaxis.axis_label = 'Count'
    p = row(p1, p2)
    ps.append(p)

In [11]:
output_file('./_includes/metadata_stats_paperfig.html')
grid = gridplot([[ps[0], ps[1]], [ps[2], ps[3]]], width=1000, height=600)
grid.spacing = 1

In [12]:
from bokeh.io import export_png
export_png(grid, filename='./_includes/metadata_stats_paperfig.png')

'/Users/matt/Documents/GRAD_SCHOOL/wearablebp.github.io/_includes/metadata_stats_paperfig.png'

# Create ED Scatter Plots and generate includes, excludes, and meet specs tables

In [13]:
meetspecs_sls = sls_save_table[(sls_save_table['Power'] > 0.98) & (sls_save_table['SBP ED'] > 2.17) & (sls_save_table['DBP ED'] > 1.39)]
meetspecs_sls['SBP ED 95CI LB'] = meetspecs_sls.apply(lambda x: np.sqrt(stats.f.isf(1-0.1, x['Number of Test Subjects']-1, x['Number of Test Subjects']-1))*x['SBP ED'], axis=1)
meetspecs_sls['DBP ED 95CI LB'] = meetspecs_sls.apply(lambda x: np.sqrt(stats.f.isf(1-0.1, x['Number of Test Subjects']-1, x['Number of Test Subjects']-1))*x['DBP ED'], axis=1)
meetspecs_cols = ['Title', 'Sensor Data', 'Calibration Technique', 'Algorithm', 'Dataset', 'SBP ED', 'DBP ED', 'SBP ED 95CI LB', 'DBP ED 95CI LB']
for col in meetspecs_sls.select_dtypes(include=['int16', 'int32', 'int64', 'float16', 'float32', 'float64']).columns:
    meetspecs_sls[col] = np.round(meetspecs_sls[col], 2)
meetspecs_sls[meetspecs_cols].to_html('./_includes/meetspecs_sls_table.html', index=False, render_links=True, table_id='mslstable', escape=False)

# per_save_table[(per_save_table['Power'] > 0.98) & (per_save_table['SBP ED'] - per_save_table['SBP ED MOE'] > 2.17) & (per_save_table['DBP ED'] - per_save_table['DBP ED MOE'] > 1.39)]

/var/folders/4k/jr4_f2px17b2sdv74jmvp4x80000gn/T/ipykernel_25023/2121789591.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meetspecs_sls['SBP ED 95CI LB'] = meetspecs_sls.apply(lambda x: np.sqrt(stats.f.isf(1-0.1, x['Number of Test Subjects']-1, x['Number of Test Subjects']-1))*x['SBP ED'], axis=1)
/var/folders/4k/jr4_f2px17b2sdv74jmvp4x80000gn/T/ipykernel_25023/2121789591.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meetspecs_sls['DBP ED 95CI LB'] = meetspecs_sls.apply(lambda x: np.sqrt(stats.f

In [14]:
# get exclude table
exclude_table = data[data['Include'] == 0]
exclude_table['Year'] = exclude_table['Year'].fillna(0)
exclude_table['Authors'] = exclude_table['Authors'].fillna('Not Available')
exclude_table['Title'] = make_clickable(exclude_table['Title'] + ' ' + exclude_table['Year'].astype(int).astype(str), exclude_table['ArticleURL'])
exclude_table['Authors'] = exclude_table['Authors'].apply(compress_authors) + ' et al.'
exclude_table = exclude_table[['Title', 'Authors', 'Exclude Reason']]
exclude_table.to_html('./_includes/exclude_table.html', index=False, render_links=True, table_id='etable', escape=False)

/var/folders/4k/jr4_f2px17b2sdv74jmvp4x80000gn/T/ipykernel_25023/2903807737.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exclude_table['Year'] = exclude_table['Year'].fillna(0)
/var/folders/4k/jr4_f2px17b2sdv74jmvp4x80000gn/T/ipykernel_25023/2903807737.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exclude_table['Authors'] = exclude_table['Authors'].fillna('Not Available')
/var/folders/4k/jr4_f2px17b2sdv74jmvp4x80000gn/T/ipykernel_25023/2903807737.py:5: SettingWithCopyWarning: 
A value is trying

In [15]:
hover = HoverTool(
        tooltips="""
        <div>
            <div>
                <span style="font-size: 20px"> @{Hover Title} </span> <br>
                <span style="font-size: 10px"> @{Title} </span> <br>
                <span style="font-size: 10px"> Sensor Data: @{Sensor Data}</span> <br>
                <span style="font-size: 10px"> SBP/DBP Reported Result (mmHg): @{SBP Reported Error Raw} /  @{DBP Reported Error Raw} </span> <br>
                <span style="font-size: 10px"> SBP Explained Deviation: @{SBP ED}</span> <br>
                <span style="font-size: 10px"> DBP Explained Deviation: @{DBP ED}</span> <br>
                <span style="font-size: 10px"> SBP Distribution STD: @{SBP Distribution STD} mmHg </span> <br>
                <span style="font-size: 10px"> DBP Distribution STD: @{DBP Distribution STD} mmHg </span> <br>
                <span style="font-size: 10px"> Power: @{Power}</span> <br>
                <span style="font-size: 10px"> Number of Test Subjects: @{Number of Test Subjects}</span> <br>
                <span style="font-size: 10px"> Study Characteristics: @{Study Characteristics}</span>
            </div>
        </div>
        """
    )
cs = ['#a6cee3', '#1f78b4', '#b2df8a', '#33a02c', '#fb9a99', '#e31a1c', '#fdbf6f', '#ff7f00', '#cab2d6', '#6a3d9a', '#ffff99', '#b15928', '#8dd3c7', '#ffffb3', '#bebada', '#fb8072', '#80b1d3', '#fdb462', '#b3de69', '#fccde5', '#d9d9d9', '#bc80bd', '#ccebc5', '#ffed6f']

In [16]:
plot_df_sls = sls_save_table[sls_save_table['Calibration Technique'] == 'subject split'].dropna(subset=['SBP ED', 'DBP ED'], axis=0)
Y = plot_df_sls['SBP ED']
X = plot_df_sls['DBP ED']
plot_df_sls_x = np.linspace(X.min(), X.max(), 100)
plot_df_sls_x = sm.add_constant(plot_df_sls_x)
X = sm.add_constant(X)

model = sm.OLS(Y,X)
results = model.fit()
sls_params = results.params.values
sls_pval = results.pvalues[1]
sls_y = results.predict(plot_df_sls_x)
sls_x = plot_df_sls_x[:, 1]
sls_ci = results.conf_int().values[1, :]

plot_df_per = per_save_table[per_save_table['Calibration Technique'] == 'personalization'].dropna(subset=['SBP ED', 'DBP ED'], axis=0)
Y = plot_df_per['SBP ED']
X = plot_df_per['DBP ED']
plot_df_per_x = np.linspace(X.min(), X.max(), 100)
plot_df_per_x = sm.add_constant(plot_df_per_x)
X = sm.add_constant(X)

model = sm.OLS(Y,X)
results = model.fit()
per_params = results.params.values
per_pval = results.pvalues[1]
per_y = results.predict(plot_df_per_x)
per_x = plot_df_per_x[:, 1]
per_ci = results.conf_int().values[1, :]

In [17]:
output_file('./_includes/scatter.html')

p1 = figure(title='Subject Split', background_fill_color="#fafafa", width=400, height=400, toolbar_location='above', tools=[hover, 'pan', 'wheel_zoom' ,'box_zoom', 'reset'])
p1.xaxis.axis_label = 'DBP Explained Deviation'
p1.yaxis.axis_label = 'SBP Explained Deviation'
p2 = figure(title='Personalization', background_fill_color="#fafafa", width=625, height=400, toolbar_location='above', tools=[hover, 'pan', 'wheel_zoom' ,'box_zoom', 'reset'])
p2.xaxis.axis_label = 'DBP Explained Deviation'
p2.yaxis.axis_label = 'SBP Explained Deviation'

box = BoxAnnotation(bottom=2.17, left=1.39, fill_alpha=0.05, fill_color='green', line_width=5, line_dash = 'dashed', line_color='green', line_alpha=0.05)
p1.add_layout(box)
p2.add_layout(box)

plot_df = df_concat.copy()

colors = {}
u_sd = plot_df['Sensor Data'].unique()
for i in range(len(u_sd)):
    colors[u_sd[i]] = cs[i]
plot_df['Color'] = [colors[i] for i in plot_df['Sensor Data']]
plot_df['Size'] = plot_df['Power']*15
plot_df['Hover Title'] = plot_df['Authors'] + ' (' + plot_df['Year'].astype(int).astype(str) + ')'

legend_it = []
for i in range(len(u_sd)):
    d1 = plot_df[plot_df['Sensor Data'] == u_sd[i]]
    sls = d1[d1['Calibration Technique'] == 'subject split']
    per = d1[d1['Calibration Technique'] == 'personalization']
    c1 = p1.scatter('DBP ED', 'SBP ED', source=ColumnDataSource(sls), fill_alpha=0.4, size='Size', color='Color', muted_alpha=0)
    c2 = p2.scatter('DBP ED', 'SBP ED', source=ColumnDataSource(per), fill_alpha=0.4, size='Size', color='Color', muted_alpha=0)
    c3 = p2.scatter('DBP ED', 'SBP ED', source=ColumnDataSource(sls), fill_alpha=0.4, size='Size', color='Color', muted_alpha=0, visible = False)
    taptool1 = p1.select(type=TapTool)
    taptool1.callback = OpenURL(url='@ArticleURL')
    taptool2 = p2.select(type=TapTool)
    taptool2.callback = OpenURL(url='@ArticleURL')
    legend_it.append((u_sd[i], [c1, c2, c3]))
    legend = Legend(items=legend_it)
    legend.click_policy='mute'

c100 = p1.line(sls_x, sls_y, line_width=3, legend_label='Slope=' + "{:.3f}".format(sls_params[1]) + ' (CI = [' + "{:.3f}".format(sls_ci[0]) + ', ' + "{:.3f}".format(sls_ci[1]) + '], ' + 'p=' + "{:.1e}".format(sls_pval) + '*)')
# c101 = p2.line(per_x, per_y, line_width=3, legend_label='Slope=' + "{:.3f}".format(per_params[1]) + ' (CI = [' + "{:.3f}".format(per_ci[0]) + ', ' + "{:.3f}".format(per_ci[1]) + '], ' + 'p=' + "{:.1e}".format(per_pval) + ')')
    
p2.add_layout(legend, 'right')
# p1.add_layout(legend, 'right')

show(row([p1, p2]))

# Create Code Availability Table

In [18]:
save_cols = ['Title', 'Authors', 'ArticleURL', 'Year', 'Sensor Data', 'Number of Test Subjects', 'Dataset', 'Calibration Technique', 'SBP ED', 'DBP ED', 'Original Model Implementation Availability', 'Original Model Implementation Link', 'Exact Dataset/Pre-processing Code Availability', 'Exact Dataset/Pre-processing Code Link', 'wearablebp.github.io Implementation Result']
availability_table = df_concat.copy()[save_cols]
availability_table['Title'] = make_clickable(availability_table['Title'] + ' (' + availability_table['Authors'] + ', ' + availability_table['Year'].astype(int).astype(str) + ')', availability_table['ArticleURL'])
# availability_table.drop('ArticleURL', axis=1, inplace=True)

availability_table['Original Model Implementation Availability'] = make_clickable(availability_table['Original Model Implementation Availability'], availability_table['Original Model Implementation Link'])
availability_table['Exact Dataset/Pre-processing Code Availability'] = make_clickable(availability_table['Exact Dataset/Pre-processing Code Availability'], availability_table['Exact Dataset/Pre-processing Code Link'])

availability_table[['Title', 'Sensor Data', 'Dataset', 'Calibration Technique', 'SBP ED', 'DBP ED', 'Original Model Implementation Availability', 'Exact Dataset/Pre-processing Code Availability', ]].rename({'SBP ED':'Reported SBP ED', 'DBP ED': 'Reported DBP ED'}, axis=1).to_html('./_includes/availability_table.html', index=False, render_links=True, table_id='atable', escape=False)

# Other paper plots

## STD vs SBP/DBP ED

Publication bias: since AAMI standards require <5±8mmHg

In [19]:
import statsmodels.api as sm
Y = plot_df[plot_df['Calibration Technique'] == 'subject split']['SBP ED']
X = plot_df[plot_df['Calibration Technique'] == 'subject split']['SBP Distribution STD']
sls_sbp_x = np.linspace(X.min(), X.max(), 100)
sls_sbp_x = sm.add_constant(sls_sbp_x)

X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
sls_sbp_params = results.params.values
sls_sbp_pval = results.pvalues[1]
sls_sbp_y = results.predict(sls_sbp_x)
sls_sbp_x = sls_sbp_x[:, 1]
sls_sbp_ci = results.conf_int().values[1, :]

dbp_plot_df = plot_df.dropna(subset=['DBP Distribution STD', 'DBP ED'], axis=0)
Y = dbp_plot_df[dbp_plot_df['Calibration Technique'] == 'subject split']['DBP ED']
X = dbp_plot_df[dbp_plot_df['Calibration Technique'] == 'subject split']['DBP Distribution STD']
sls_dbp_x = np.linspace(X.min(), X.max(), 100)
sls_dbp_x = sm.add_constant(sls_dbp_x)

X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
sls_dbp_params = results.params.values
sls_dbp_pval = results.pvalues[1]
sls_dbp_y = results.predict(sls_dbp_x)
sls_dbp_x = sls_dbp_x[:, 1]
sls_dbp_ci = results.conf_int().values[1, :]

Y = plot_df[plot_df['Calibration Technique'] == 'personalization']['SBP ED']
X = plot_df[plot_df['Calibration Technique'] == 'personalization']['SBP Distribution STD']
per_sbp_x = np.linspace(X.min(), X.max(), 100)
per_sbp_x = sm.add_constant(per_sbp_x)

X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
per_sbp_params = results.params.values
per_sbp_pval = results.pvalues[1]
per_sbp_y = results.predict(per_sbp_x)
per_sbp_x = per_sbp_x[:, 1]
per_sbp_ci = results.conf_int().values[1, :]

dbp_plot_df = plot_df.dropna(subset=['DBP Distribution STD', 'DBP ED'], axis=0)
Y = dbp_plot_df[dbp_plot_df['Calibration Technique'] == 'personalization']['DBP ED']
X = dbp_plot_df[dbp_plot_df['Calibration Technique'] == 'personalization']['DBP Distribution STD']
per_dbp_x = np.linspace(X.min(), X.max(), 100)
per_dbp_x = sm.add_constant(per_dbp_x)

X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
per_dbp_params = results.params.values
per_dbp_pval = results.pvalues[1]
per_dbp_y = results.predict(per_dbp_x)
per_dbp_x = per_dbp_x[:, 1]
per_dbp_ci = results.conf_int().values[1, :]

In [20]:
output_file('./_includes/scatter_std_ed.html')

hover = HoverTool(
        tooltips="""
        <div>
            <div>
                <span style="font-size: 20px"> @{Hover Title} </span> <br>
                <span style="font-size: 10px"> @{Title} </span> <br>
                <span style="font-size: 10px"> Sensor Data: @{Sensor Data}</span> <br>
                <span style="font-size: 10px"> SBP/DBP Reported Result (mmHg): @{SBP Reported Error Raw} /  @{DBP Reported Error Raw} </span> <br>
                <span style="font-size: 10px"> SBP Explained Deviation: @{SBP ED}</span> <br>
                <span style="font-size: 10px"> DBP Explained Deviation: @{DBP ED}</span> <br>
                <span style="font-size: 10px"> SBP Distribution STD: @{SBP Distribution STD} mmHg </span> <br>
                <span style="font-size: 10px"> DBP Distribution STD: @{DBP Distribution STD} mmHg </span> <br>
                <span style="font-size: 10px"> Power: @{Power}</span> <br>
                <span style="font-size: 10px"> Number of Test Subjects: @{Number of Test Subjects}</span> <br>
                <span style="font-size: 10px"> Study Characteristics: @{Study Characteristics}</span>
            </div>
        </div>
        """
    )
cs = ['#a6cee3', '#1f78b4', '#b2df8a', '#33a02c', '#fb9a99', '#e31a1c', '#fdbf6f', '#ff7f00', '#cab2d6', '#6a3d9a', '#ffff99', '#b15928', '#8dd3c7', '#ffffb3', '#bebada', '#fb8072', '#80b1d3', '#fdb462', '#b3de69', '#fccde5', '#d9d9d9', '#bc80bd', '#ccebc5', '#ffed6f']

p3 = figure(title='Subject Split', background_fill_color="#fafafa", width=400, height=400, toolbar_location='above', tools=[hover, 'pan', 'wheel_zoom' ,'box_zoom', 'reset'])
p3.xaxis.axis_label = 'SBP Distribution STD (mmHg)'
p3.yaxis.axis_label = 'SBP Explained Deviation'
p4 = figure(title='Personalization', background_fill_color="#fafafa", width=645, height=400, toolbar_location='above', tools=[hover, 'pan', 'wheel_zoom' ,'box_zoom', 'reset'])
p4.xaxis.axis_label = 'SBP Distribution STD (mmHg)'
p4.yaxis.axis_label = ''

p5 = figure(title='', background_fill_color="#fafafa", width=400, height=400, toolbar_location='above', tools=[hover, 'pan', 'wheel_zoom' ,'box_zoom', 'reset'])
p5.xaxis.axis_label = 'DBP Distribution STD (mmHg)'
p5.yaxis.axis_label = 'DBP Explained Deviation'
p6 = figure(title='', background_fill_color="#fafafa", width=400, height=400, toolbar_location='above', tools=[hover, 'pan', 'wheel_zoom' ,'box_zoom', 'reset'])
p6.xaxis.axis_label = 'DBP Distribution STD (mmHg)'
p6.yaxis.axis_label = ''

box1 = BoxAnnotation(bottom=2.17, left=17.36, fill_alpha=0.05, fill_color='green', line_width=5, line_dash = 'dashed', line_color='green', line_alpha=0.05)
box2 = BoxAnnotation(bottom=1.39, left=11.09, fill_alpha=0.05, fill_color='green', line_width=5, line_dash = 'dashed', line_color='green', line_alpha=0.05)
p3.add_layout(box1)
p4.add_layout(box1)
p5.add_layout(box2)
p6.add_layout(box2)

plot_df = df_concat.copy()

colors = {}
u_sd = plot_df['Sensor Data'].unique()
for i in range(len(u_sd)):
    colors[u_sd[i]] = cs[i]
plot_df['Color'] = [colors[i] for i in plot_df['Sensor Data']]
plot_df['Size'] = plot_df['Power']*15
plot_df['Hover Title'] = plot_df['Authors'] + ' (' + plot_df['Year'].astype(int).astype(str) + ')'

legend_it1 = []
legend_it2 = []
for i in range(len(u_sd)):
    d1 = plot_df[plot_df['Sensor Data'] == u_sd[i]]
    sls = d1[d1['Calibration Technique'] == 'subject split']
    per = d1[d1['Calibration Technique'] == 'personalization']
    c4 = p3.scatter('SBP Distribution STD', 'SBP ED', source=ColumnDataSource(sls), fill_alpha=0.4, size='Size', color='Color', muted_alpha=0)
    c5 = p4.scatter('SBP Distribution STD', 'SBP ED', source=ColumnDataSource(per), fill_alpha=0.4, size='Size', color='Color', muted_alpha=0)
    c6 = p4.scatter('SBP Distribution STD', 'SBP ED', source=ColumnDataSource(sls), fill_alpha=0.4, size='Size', color='Color', muted_alpha=0, visible = False)
    taptool1 = p3.select(type=TapTool)
    taptool1.callback = OpenURL(url='@ArticleURL')
    taptool2 = p4.select(type=TapTool)
    taptool2.callback = OpenURL(url='@ArticleURL')
    
    c7 = p5.scatter('DBP Distribution STD', 'DBP ED', source=ColumnDataSource(sls), fill_alpha=0.4, size='Size', color='Color', muted_alpha=0)
    c8 = p6.scatter('DBP Distribution STD', 'DBP ED', source=ColumnDataSource(per), fill_alpha=0.4, size='Size', color='Color', muted_alpha=0)
    c9 = p6.scatter('DBP Distribution STD', 'DBP ED', source=ColumnDataSource(sls), fill_alpha=0.4, size='Size', color='Color', muted_alpha=0, visible = False)
    taptool3 = p5.select(type=TapTool)
    taptool3.callback = OpenURL(url='@ArticleURL')
    taptool4 = p6.select(type=TapTool)
    taptool4.callback = OpenURL(url='@ArticleURL')
    
    legend_it1.append((u_sd[i], [c4, c5, c6, c7, c8, c9]))
    legend1 = Legend(items=legend_it1)
    legend1.click_policy='mute'

c10 = p3.line(sls_sbp_x, sls_sbp_y, line_width=3, legend_label='Slope=' + "{:.3f}".format(sls_sbp_params[1]) + '(CI = [' + "{:.3f}".format(sls_sbp_ci[0]) + ', ' + "{:.3f}".format(sls_sbp_ci[1]) + '], ' +'p=' + "{:.1e}".format(sls_sbp_pval) + '*)')
c11 = p4.line(per_sbp_x, per_sbp_y, line_width=3, legend_label='Slope=' + "{:.3f}".format(per_sbp_params[1]) + ' (CI = [' + "{:.3f}".format(per_sbp_ci[0]) + ', ' + "{:.3f}".format(per_sbp_ci[1]) + '], ' + 'p=' + "{:.1e}".format(per_sbp_pval) + '*)')
c12 = p5.line(sls_dbp_x, sls_dbp_y, line_width=3, legend_label='Slope=' + "{:.3f}".format(sls_dbp_params[1]) + ' (CI = [' + "{:.3f}".format(sls_dbp_ci[0]) + ', ' + "{:.3f}".format(sls_dbp_ci[1]) + '], ' + 'p=' + "{:.1e}".format(sls_dbp_pval) + '*)')
c13 = p6.line(per_dbp_x, per_dbp_y, line_width=3, legend_label='Slope=' + "{:.3f}".format(per_dbp_params[1]) + ' (CI = [' + "{:.3f}".format(per_dbp_ci[0]) + ', ' + "{:.3f}".format(per_dbp_ci[1]) + '], ' + 'p=' + "{:.1e}".format(per_dbp_pval) + '*)')
    
p4.add_layout(legend1, 'right')
# p6.add_layout(legend2, 'right')
# p1.add_layout(legend, 'right')


show(column(row([p3, p4]), row([p5, p6])))

## Error vs STD

In [21]:
Y = plot_df[plot_df['Calibration Technique'] == 'subject split']['SBP Error STD']
X = plot_df[plot_df['Calibration Technique'] == 'subject split']['SBP Distribution STD']
sls_sbp_x = np.linspace(X.min(), X.max(), 100)
sls_sbp_x = sm.add_constant(sls_sbp_x)
X = sm.add_constant(X)

model = sm.OLS(Y,X)
results = model.fit()
sls_sbp_params = results.params.values
sls_sbp_pval = results.pvalues[1]
sls_sbp_y = results.predict(sls_sbp_x)
sls_sbp_x = sls_sbp_x[:, 1]
sls_sbp_ci = results.conf_int().values[1, :]

dbp_plot_df = plot_df.dropna(subset=['DBP Distribution STD', 'DBP Error STD'], axis=0)
Y = dbp_plot_df[dbp_plot_df['Calibration Technique'] == 'subject split']['DBP Error STD']
X = dbp_plot_df[dbp_plot_df['Calibration Technique'] == 'subject split']['DBP Distribution STD']
sls_dbp_x = np.linspace(X.min(), X.max(), 100)
sls_dbp_x = sm.add_constant(sls_dbp_x)
X = sm.add_constant(X)

model = sm.OLS(Y,X)
results = model.fit()
sls_dbp_params = results.params.values
sls_dbp_pval = results.pvalues[1]
sls_dbp_y = results.predict(sls_dbp_x)
sls_dbp_x = sls_dbp_x[:, 1]
sls_dbp_ci = results.conf_int().values[1, :]

Y = plot_df[plot_df['Calibration Technique'] == 'personalization']['SBP Error STD']
X = plot_df[plot_df['Calibration Technique'] == 'personalization']['SBP Distribution STD']
per_sbp_x = np.linspace(X.min(), X.max(), 100)
per_sbp_x = sm.add_constant(per_sbp_x)
X = sm.add_constant(X)

model = sm.OLS(Y,X)
results = model.fit()
per_sbp_params = results.params.values
per_sbp_pval = results.pvalues[1]
per_sbp_y = results.predict(per_sbp_x)
per_sbp_x = per_sbp_x[:, 1]
per_sbp_ci = results.conf_int().values[1, :]

dbp_plot_df = plot_df.dropna(subset=['DBP Distribution STD', 'DBP Error STD'], axis=0)
Y = dbp_plot_df[dbp_plot_df['Calibration Technique'] == 'personalization']['DBP Error STD']
X = dbp_plot_df[dbp_plot_df['Calibration Technique'] == 'personalization']['DBP Distribution STD']
per_dbp_x = np.linspace(X.min(), X.max(), 100)
per_dbp_x = sm.add_constant(per_dbp_x)
X = sm.add_constant(X)

model = sm.OLS(Y,X)
results = model.fit()
per_dbp_params = results.params.values
per_dbp_pval = results.pvalues[1]
per_dbp_y = results.predict(per_dbp_x)
per_dbp_x = per_dbp_x[:, 1]
per_dbp_ci = results.conf_int().values[1, :]

In [34]:
hover = HoverTool(
        tooltips="""
        <div>
            <div>
                <span style="font-size: 20px"> @{Hover Title} </span> <br>
                <span style="font-size: 10px"> @{Title} </span> <br>
                <span style="font-size: 10px"> Sensor Data: @{Sensor Data}</span> <br>
                <span style="font-size: 10px"> SBP/DBP Reported Result (mmHg): @{SBP Reported Error Raw} /  @{DBP Reported Error Raw} </span> <br>
                <span style="font-size: 10px"> SBP Explained Deviation: @{SBP ED}</span> <br>
                <span style="font-size: 10px"> DBP Explained Deviation: @{DBP ED}</span> <br>
                <span style="font-size: 10px"> SBP Distribution STD: @{SBP Distribution STD} mmHg </span> <br>
                <span style="font-size: 10px"> DBP Distribution STD: @{DBP Distribution STD} mmHg </span> <br>
                <span style="font-size: 10px"> Power: @{Power}</span> <br>
                <span style="font-size: 10px"> Number of Test Subjects: @{Number of Test Subjects}</span> <br>
                <span style="font-size: 10px"> Study Characteristics: @{Study Characteristics}</span>
            </div>
        </div>
        """
    )
cs = ['#a6cee3', '#1f78b4', '#b2df8a', '#33a02c', '#fb9a99', '#e31a1c', '#fdbf6f', '#ff7f00', '#cab2d6', '#6a3d9a', '#ffff99', '#b15928', '#ffffb3', '#bebada', '#fb8072', '#80b1d3', '#fdb462', '#b3de69', '#fccde5', '#d9d9d9', '#bc80bd', '#ccebc5', '#ffed6f']

p203 = figure(title='Subject Split', background_fill_color="#fafafa", width=400, height=400, toolbar_location='above', tools=[hover, 'pan', 'wheel_zoom' ,'box_zoom', 'reset'])
p203.xaxis.axis_label = 'SBP Distribution STD (mmHg)'
p203.yaxis.axis_label = 'SBP Error STD (mmHg)'
# p4 = figure(title='Personalization', background_fill_color="#fafafa", width=645, height=400, toolbar_location='above', tools=[hover, 'pan', 'wheel_zoom' ,'box_zoom', 'reset'])
# p4.xaxis.axis_label = 'SBP Distribution STD (mmHg)'
# p4.yaxis.axis_label = ''

# p205 = figure(title='', background_fill_color="#fafafa", width=400, height=400, toolbar_location='above', tools=[hover, 'pan', 'wheel_zoom' ,'box_zoom', 'reset'])
p205 = figure(title='', background_fill_color="#fafafa", width=625, height=400, toolbar_location='above', tools=[hover, 'pan', 'wheel_zoom' ,'box_zoom', 'reset'])
p205.xaxis.axis_label = 'DBP Distribution STD (mmHg)'
p205.yaxis.axis_label = 'DBP Error STD (mmHg)'
# p6 = figure(title='', background_fill_color="#fafafa", width=400, height=400, toolbar_location='above', tools=[hover, 'pan', 'wheel_zoom' ,'box_zoom', 'reset'])
# p6.xaxis.axis_label = 'DBP Distribution STD (mmHg)'
# p6.yaxis.axis_label = ''

# box1 = BoxAnnotation(top=8, left=17.36, fill_alpha=0.05, fill_color='green', line_width=5, line_dash = 'dashed', line_color='green', line_alpha=0.05)
# box2 = BoxAnnotation(top=8, left=11.09, fill_alpha=0.05, fill_color='green', line_width=5, line_dash = 'dashed', line_color='green', line_alpha=0.05)
# p203.add_layout(box1)
# p4.add_layout(box1)
# p205.add_layout(box2)
# p6.add_layout(box2)

plot_df = df_concat.copy()

colors = {}
u_sd = plot_df['Sensor Data'].unique()
for i in range(len(u_sd)):
    colors[u_sd[i]] = cs[i]
plot_df['Color'] = [colors[i] for i in plot_df['Sensor Data']]
plot_df['Size'] = plot_df['Power']*15
plot_df['Hover Title'] = plot_df['Authors'] + ' (' + plot_df['Year'].astype(int).astype(str) + ')'

legend_it201 = []
for i in range(len(u_sd)):
    d1 = plot_df[plot_df['Sensor Data'] == u_sd[i]]
    sls = d1[d1['Calibration Technique'] == 'subject split'].dropna(subset=['Sensor Data'])
    per = d1[d1['Calibration Technique'] == 'personalization']
    if len(sls) > 0:
        print(u_sd[i], len(sls))
        if u_sd[i] == 'ECG+PCG+PPG':
            print(sls[['SBP Distribution STD', 'SBP Error STD']])
        c204 = p203.scatter('SBP Distribution STD', 'SBP Error STD', source=ColumnDataSource(sls), fill_alpha=0.4, size='Size', color='Color', muted_alpha=0)
        # c5 = p4.scatter('SBP Distribution STD', 'SBP Error STD', source=ColumnDataSource(per), fill_alpha=0.4, size='Size', color='Color', muted_alpha=0)
        # c6 = p4.scatter('SBP Distribution STD', 'SBP Error STD', source=ColumnDataSource(sls), fill_alpha=0.4, size='Size', color='Color', muted_alpha=0, visible = False)
        taptool1 = p203.select(type=TapTool)
        taptool1.callback = OpenURL(url='@ArticleURL')
        # taptool2 = p4.select(type=TapTool)
        # taptool2.callback = OpenURL(url='@ArticleURL')

        c207 = p205.scatter('DBP Distribution STD', 'DBP Error STD', source=ColumnDataSource(sls), fill_alpha=0.4, size='Size', color='Color', muted_alpha=0)
        # c8 = p6.scatter('DBP Distribution STD', 'DBP Error STD', source=ColumnDataSource(per), fill_alpha=0.4, size='Size', color='Color', muted_alpha=0)
        # c9 = p6.scatter('DBP Distribution STD', 'DBP Error STD', source=ColumnDataSource(sls), fill_alpha=0.4, size='Size', color='Color', muted_alpha=0, visible = False)
        taptool3 = p205.select(type=TapTool)
        taptool3.callback = OpenURL(url='@ArticleURL')
        # taptool4 = p6.select(type=TapTool)
        # taptool4.callback = OpenURL(url='@ArticleURL')

        # legend_it201.append((u_sd[i], [c204, c5, c6, c207, c8, c9]))
        legend_it201.append((u_sd[i], [c204, c207]))
        legend201 = Legend(items=legend_it201)
        legend201.click_policy='mute'
    
c10 = p203.line(sls_sbp_x, sls_sbp_y, line_width=3, legend_label='Slope=' + "{:.3f}".format(sls_sbp_params[1]) + '(CI = [' + "{:.3f}".format(sls_sbp_ci[0]) + ', ' + "{:.3f}".format(sls_sbp_ci[1]) + '], ' +'p=' + "{:.1e}".format(sls_sbp_pval) + '*)')
# c11 = p4.line(per_sbp_x, per_sbp_y, line_width=3, legend_label='Slope=' + "{:.3f}".format(per_sbp_params[1]) + ' (CI = [' + "{:.3f}".format(per_sbp_ci[0]) + ', ' + "{:.3f}".format(per_sbp_ci[1]) + '], ' + 'p=' + "{:.1e}".format(per_sbp_pval) + ')')
c12 = p205.line(sls_dbp_x, sls_dbp_y, line_width=3, legend_label='Slope=' + "{:.3f}".format(sls_dbp_params[1]) + ' (CI = [' + "{:.3f}".format(sls_dbp_ci[0]) + ', ' + "{:.3f}".format(sls_dbp_ci[1]) + '], ' + 'p=' + "{:.1e}".format(sls_dbp_pval) + '*)')
# c13 = p6.line(per_dbp_x, per_dbp_y, line_width=3, legend_label='Slope=' + "{:.3f}".format(per_dbp_params[1]) + ' (CI = [' + "{:.3f}".format(per_dbp_ci[0]) + ', ' + "{:.3f}".format(per_dbp_ci[1]) + '], ' + 'p=' + "{:.1e}".format(per_dbp_pval) + ')')
    
p205.add_layout(legend201, 'right')
# p6.add_layout(legend2, 'right')
# p1.add_layout(legend, 'right')


# show(row([p203, p205]), row([p205, p6])))
output_file('./_includes/scatter_std_err.html')
show(row([p203, p205]))

ECG 4
ECG+PPG 15
PPG+PPG 5
BCG+PPG 1
PPG 15
PPG+pressure sensor 1
biometrics+pressure 1
pressure+multi-wavelength PPG 1
biometrics+PPG 3
pressure+PPG 2
ECG+PCG+PPG 3
      SBP Distribution STD  SBP Error STD
1539                  7.95            6.6
1630                 20.00            6.3
1631                 12.90            6.3
pressure sensor 1
biometrics+ECG+PPG 1
ICG 2
ultrasound 2
biometrics+PCG 1
PPG+SCG 1
ECG+ICG+PPG 1


## Time vs Accuracy Scatter Plot

In [ ]:
def convert_dt(x):
    if 's' in x:
        if x.split('s')[0] == '0':
            return 0
        return np.log10(int(x.split('s')[0]))+0
    elif 'm' in x:
        return np.log10(int(x.split('m')[0]))+1
    elif 'h' in x:
        return np.log10(int(x.split('h')[0]))+2
    elif 'd' in x:
        return np.log10(int(x.split('d')[0]))+3
    elif 'mon' in x:
        return np.log10(int(x.split('mon')[0]))+4

per = data[(data['Include'] == 1) & (data['Study Characteristics'].str.contains('observational')) & (data['Time between Calibration and Test'].str.contains('unclear') == False) & (data['Time between Calibration and Test'].str.contains('heart beats') == False) & ((data['Evaluation Metric'] == 'MAE') | (data['Evaluation Metric'] == 'ME'))]
per['Calibration Technique'] = per['Calibration Technique'].str.split(';', expand=True)[0]
per = per[per['Calibration Technique'] == 'personalization']
per['log_dt'] = per['Time between Calibration and Test'].apply(convert_dt)

per['Sensor Data'] = per['Key Devices and Measurements'].str.split(';', expand=True)[0]
per['Calibration Technique'] = per['Calibration Technique'].str.split(';', expand=True)[0]
per['Algorithm'] = per['Algorithm'].str.split(';', expand=True)[0]
per['Dataset'] = per['Dataset'].str.split(';', expand=True)[0]
per['Testing Subject Characteristics'] = per['Testing Subject Characteristics'].str.split(';', expand=True)[0]
per['Study Characteristics'] = per['Study Characteristics'].str.split(';', expand=True)[0]
per['Algorithm'] = per['Algorithm'].str.split(';', expand=True)[0]
per['SBP Distribution Raw'] = per['BP Distribution'].str.split(';', expand=True)[0]
per['SBP Distribution STD'] = per['SBP Distribution Raw'].str.split('±', expand=True)[1].astype(float)
per['DBP Distribution Raw'] = per['BP Distribution'].str.split(';', expand=True)[1]
per['DBP Distribution STD'] = per['DBP Distribution Raw'].str.split('±', expand=True)[1].astype(float)
per['SBP Reported Error Raw'] = per['Reported Result'].str.split(';', expand=True)[0]
per['DBP Reported Error Raw'] = per['Reported Result'].str.split(';', expand=True)[1]

per_mae = per[per['Evaluation Metric'] == 'MAE']
per_mae['SBP Error STD'] = per_mae['SBP Reported Error Raw'].astype(float)*np.sqrt(pi/2)
per_mae['DBP Error STD'] = per_mae['DBP Reported Error Raw'].astype(float)*np.sqrt(pi/2)
per_me = per[per['Evaluation Metric'] == 'ME']
per_me['SBP Error STD'] = per_me['SBP Reported Error Raw'].str.split('±', expand=True)[1].astype(float)
per_me['DBP Error STD'] = per_me['DBP Reported Error Raw'].str.split('±', expand=True)[1].astype(float)
per_concat = pd.concat([per_me, per_mae], axis=0)
per_concat['SBP ED'] = per_concat['SBP Distribution STD']/per_concat['SBP Error STD']
per_concat['DBP ED'] = per_concat['DBP Distribution STD']/per_concat['DBP Error STD']

per_concat['Number of Test Subjects'] = per_concat['Number of Test Subjects'].replace({'unclear': np.NaN})
per_concat = per_concat.dropna(subset=['Number of Test Subjects'])
per_concat['Power'] = per_concat['Number of Test Subjects'].astype(float).apply(compute_power)

In [ ]:
per_concat_sbp = per_concat.dropna(subset=['SBP Error STD', 'log_dt'], axis=0)
per_concat_sbp = per_concat_sbp[per_concat_sbp['log_dt'] > 0]
Y = per_concat_sbp['SBP Error STD']
X = per_concat_sbp['log_dt']
per_sbp_x = np.linspace(X.min(), X.max(), 100)
per_sbp_x = sm.add_constant(per_sbp_x)
X = sm.add_constant(X)

model = sm.OLS(Y,X)
results = model.fit()
per_sbp_params = results.params.values
per_sbp_pval = results.pvalues[1]
per_sbp_y = results.predict(per_sbp_x)
per_sbp_x = per_sbp_x[:, 1]
per_sbp_ci = results.conf_int().values[1, :]

per_concat_dbp = per_concat.dropna(subset=['DBP Error STD', 'log_dt'], axis=0)
per_concat_dbp = per_concat_dbp[per_concat_dbp['log_dt'] > 0]
Y = per_concat_dbp['DBP Error STD']
X = per_concat_dbp['log_dt']
per_dbp_x = np.linspace(X.min(), X.max(), 100)
per_dbp_x = sm.add_constant(per_dbp_x)
X = sm.add_constant(X)

model = sm.OLS(Y,X)
results = model.fit()
per_dbp_params = results.params.values
per_dbp_pval = results.pvalues[1]
per_dbp_y = results.predict(per_dbp_x)
per_dbp_x = per_dbp_x[:, 1]
per_dbp_ci = results.conf_int().values[1, :]

In [ ]:
output_file('./_includes/scatter_time.html')

hover = HoverTool(
        tooltips="""
        <div>
            <div>
                <span style="font-size: 20px"> @{Hover Title} </span> <br>
                <span style="font-size: 10px"> @{Title} </span> <br>
                <span style="font-size: 10px"> Sensor Data: @{Sensor Data}</span> <br>
                <span style="font-size: 10px"> SBP/DBP Reported Result (mmHg): @{SBP Reported Error Raw} /  @{DBP Reported Error Raw} </span> <br>
                <span style="font-size: 10px"> SBP Explained Deviation: @{SBP ED}</span> <br>
                <span style="font-size: 10px"> DBP Explained Deviation: @{DBP ED}</span> <br>
                <span style="font-size: 10px"> SBP Distribution STD: @{SBP Distribution STD} mmHg </span> <br>
                <span style="font-size: 10px"> DBP Distribution STD: @{DBP Distribution STD} mmHg </span> <br>
                <span style="font-size: 10px"> Power: @{Power}</span> <br>
                <span style="font-size: 10px"> Number of Test Subjects: @{Number of Test Subjects}</span> <br>
                <span style="font-size: 10px"> Study Characteristics: @{Study Characteristics}</span>
            </div>
        </div>
        """
    )
cs = ['#a6cee3', '#1f78b4', '#b2df8a', '#33a02c', '#fb9a99', '#e31a1c', '#fdbf6f', '#ff7f00', '#cab2d6', '#6a3d9a', '#ffff99', '#b15928', '#8dd3c7', '#ffffb3', '#bebada', '#fb8072', '#80b1d3', '#fdb462', '#b3de69', '#fccde5', '#d9d9d9', '#bc80bd', '#ccebc5', '#ffed6f']

p10 = figure(title='', background_fill_color="#fafafa", width=400, height=400, toolbar_location='above', tools=[hover, 'pan', 'wheel_zoom' ,'box_zoom', 'reset'])
p10.xaxis.axis_label = 'Time between calibration and test'
p10.yaxis.axis_label = 'SBP Error STD (mmHg)'

p11 = figure(title='', background_fill_color="#fafafa", width=645, height=400, toolbar_location='above', tools=[hover, 'pan', 'wheel_zoom' ,'box_zoom', 'reset'])
p11.xaxis.axis_label = 'Time between calibration and test'
p11.yaxis.axis_label = 'DBP Error STD (mmHg)'

plot_df = per_concat.copy()

colors = {}
u_sd = plot_df['Sensor Data'].unique()
for i in range(len(u_sd)):
    colors[u_sd[i]] = cs[i]
plot_df['Color'] = [colors[i] for i in plot_df['Sensor Data']]
plot_df['Size'] = plot_df['Power']*15
plot_df['Hover Title'] = plot_df['Authors'] + ' (' + plot_df['Year'].astype(int).astype(str) + ')'

legend_it1 = []
for i in range(len(u_sd)):
    d1 = plot_df[plot_df['Sensor Data'] == u_sd[i]]
    per = d1[d1['Calibration Technique'] == 'personalization']
    c4 = p10.scatter('log_dt', 'SBP Error STD', source=ColumnDataSource(per), fill_alpha=0.4, size='Size', color='Color', muted_alpha=0)
    c5 = p11.scatter('log_dt', 'SBP Error STD', source=ColumnDataSource(per), fill_alpha=0.4, size='Size', color='Color', muted_alpha=0)
    c6 = p11.scatter('log_dt', 'DBP Error STD', source=ColumnDataSource(per), fill_alpha=0.4, size='Size', color='Color', muted_alpha=0, visible = False)
    taptool1 = p10.select(type=TapTool)
    taptool1.callback = OpenURL(url='@ArticleURL')
    taptool2 = p11.select(type=TapTool)
    taptool2.callback = OpenURL(url='@ArticleURL')

    legend_it1.append((u_sd[i], [c4, c5, c6]))
    legend1 = Legend(items=legend_it1)
    legend1.click_policy='mute'

# c10 = p10.line(per_sbp_x, per_sbp_y, line_width=3, legend_label='Slope=' + "{:.3f}".format(per_sbp_params[1]) + ' (CI = [' + "{:.3f}".format(per_sbp_ci[0]) + ', ' + "{:.3f}".format(per_sbp_ci[1]) + '], ' + 'p=' + "{:.1e}".format(per_sbp_pval) + ')')
# c11 = p11.line(per_dbp_x, per_dbp_y, line_width=3, legend_label='Slope=' + "{:.3f}".format(per_dbp_params[1]) + ' (CI = [' + "{:.3f}".format(per_dbp_ci[0]) + ', ' + "{:.3f}".format(per_dbp_ci[1]) + '], ' + 'p=' + "{:.1e}".format(per_dbp_pval) + ')')
    
p11.add_layout(legend1, 'right')
# p6.add_layout(legend2, 'right')
# p1.add_layout(legend, 'right')

p10.xaxis.ticker = [0, 1, 2, 3, 4]
p10.xaxis.major_label_overrides = {0: 'seconds', 1: 'minutes', 2: 'hours', 3: 'days', 4:'months'}
p11.xaxis.ticker = [0, 1, 2, 3, 4]
p11.xaxis.major_label_overrides = {0: 'seconds', 1: 'minutes', 2: 'hours', 3: 'days', 4:'months'}

show(row([p10, p11]))

# Supplementary

stratify: calibration technique, algorithm, Dataset, Testing Subject Characteristics, Study Characteristics, Sensor Data

regress: Number of Test Subjects, Power, SBP Distribution STD, DBP Distribution STD, SBP Error STD, DBP Error STD, SBP ED, DBP ED

find significant relationships

In [ ]:
import statsmodels.formula.api as smf

supp_df = df_concat.copy()
supp_df.columns = [x.replace(' ', '_') for x in supp_df.columns.values]
cats = ['Calibration Technique', 'Algorithm', 'Dataset', 'Testing Subject Characteristics', 'Sensor Data']
regs = ['Power', 'SBP Distribution STD', 'DBP Distribution STD', 'SBP Error STD', 'DBP Error STD', 'SBP ED', 'DBP ED']
cats = [x.replace(' ', '_') for x in cats]
regs = [x.replace(' ', '_') for x in regs]

corr = supp_df[regs].corr() 
ax = plt.axes()
sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns)
ax.set_title('Correlation')

def regress(reg1, reg2, cats, df):
    
    if cats == None:
        d = df[[reg1, reg2]].dropna()
        model = smf.ols(formula=reg1+'~'+reg2, data=d)
        results = model.fit()
        param = results.params[1]
        pval = results.pvalues[1]
        ci = results.conf_int().values[1, :]
        sig = pval < 0.05
    else:
        cols = [reg1, reg2]
        f = reg1+'~'+reg2
        for cat in cats:
            cols.append(cat)
            f+='+ C('+cat+')'
        if len(cats) == 1:
            cats = cats[0]
        d = df[cols].dropna()
        model = smf.ols(formula=f, data=d)
        results = model.fit()
        param = results.params.values[1:]
        pval = results.pvalues.values[1:]
        ci = results.conf_int().values[1:, :]
        sig = (pval < 0.05).all()
    return results, pd.DataFrame([reg1, reg2, cats, param, pval, ci, sig]).T
   
    

df_search = pd.DataFrame()
plotted_pairs = []
for reg1 in regs:
    for reg2 in regs:
        if (reg1 != reg2) & ((reg2, reg1) not in plotted_pairs):
            plotted_pairs.append((reg1, reg2))
            _, res = regress(reg1, reg2, None, supp_df)
            df_search = pd.concat([df_search, res])
            for cat in cats:
                _, res2 = regress(reg1, reg2, [cat], supp_df)
                df_search = pd.concat([df_search, res2])
df_search.columns = ['Reg1', 'Reg2', 'Cat', 'Slope', 'p-value', 'CI', 'Significant']
df_search.reset_index(drop=True, inplace=True)
df_search.to_csv('supplementary/regression_results.csv', index=False)
df_search[df_search.Significant].to_csv('supplementary/significant_regression_results.csv', index=False)
df_search[df_search.Significant]

In [ ]:
res, _ = regress('DBP_ED', 'SBP_ED', ['Testing_Subject_Characteristics'], supp_df)
res.summary()
# between categories not significant

Plots that make sense:

1. SBP/DBP Distribution STD vs Power
2. SBP Error STD vs DBP Error STD

In [ ]:
res_s, _ = regress('SBP_Distribution_STD', 'Power', None, supp_df)
res_d, _ = regress('DBP_Distribution_STD', 'Power', None, supp_df)

fig, ax = plt.subplots(1, 2, figsize=(15,5))    
ax[0].scatter(supp_df['Power'], supp_df['SBP_Distribution_STD'])
ax[0].set_xlabel('Power')
ax[0].set_ylabel('SBP Distribution STD (mmHg)')
plot_x = np.linspace(supp_df['Power'].min(), supp_df['Power'].max(), 100)
ax[0].plot(plot_x, res_s.params.values[0] + res_s.params.values[1]*plot_x, linestyle='--', c='r', label='Slope='+"{:.3f}".format(res_s.params.values[1])+', CI = ['+"{:.3f}".format(res_s.conf_int().values[1:, 0][0])+', '+"{:.3f}".format(res_s.conf_int().values[1:, 1][0])+'], p='+"{:.3e}".format(res_s.pvalues.values[1]))
ax[0].legend()
ax[1].scatter(supp_df['Power'], supp_df['DBP_Distribution_STD'])
ax[1].set_xlabel('Power')
ax[1].set_ylabel('DBP Error STD (mmHg)')
ax[1].plot(plot_x, res_d.params.values[0] + res_d.params.values[1]*plot_x, linestyle='--', c='r', label='Slope='+"{:.3f}".format(res_d.params.values[1])+', CI = ['+"{:.3f}".format(res_d.conf_int().values[1:, 0][0])+', '+"{:.3f}".format(res_d.conf_int().values[1:, 1][0])+'], p='+"{:.3e}".format(res_d.pvalues.values[1]))
ax[1].legend()
# res_s.summary(), res_d.summary()

In [ ]:
res, _ = regress('SBP_Error_STD', 'DBP_Error_STD', None, supp_df)
plt.scatter(supp_df['DBP_Error_STD'], supp_df['SBP_Error_STD'])
plot_x = np.linspace(supp_df['DBP_Error_STD'].min(), supp_df['DBP_Error_STD'].max(), 100)
plt.plot(plot_x, res.params.values[0] + res.params.values[1]*plot_x, linestyle='--', c='r', label='Slope='+"{:.3f}".format(res.params.values[1])+', CI = ['+"{:.3f}".format(res.conf_int().values[1:, 0][0])+', '+"{:.3f}".format(res.conf_int().values[1:, 1][0])+'], p='+"{:.3e}".format(res.pvalues.values[1]))
plt.legend()
plt.xlabel('DBP Error STD')
plt.ylabel('SBP Error STD')

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15, 5))
ax[0].scatter(per_concat.log_dt, per_concat['SBP Distribution STD'])
ax[1].scatter(per_concat.log_dt, per_concat['DBP Distribution STD'])

temp_df = per_concat.copy()
temp_df.columns = [x.replace(' ', '_') for x in temp_df.columns]
model = smf.ols(formula='SBP_Distribution_STD ~ log_dt', data=temp_df)
res_s = model.fit()
model = smf.ols(formula='DBP_Distribution_STD ~ log_dt', data=temp_df)
res_d = model.fit()

plot_xs = np.linspace(temp_df['log_dt'].min(), temp_df['log_dt'].max(), 100)
plot_xd = np.linspace(temp_df['log_dt'].min(), temp_df['log_dt'].max(), 100)
ax[0].plot(plot_xs, res_s.params.values[0] + res_s.params.values[1]*plot_xs, linestyle='--', c='r', label='Slope='+"{:.3f}".format(res_s.params.values[1])+', CI = ['+"{:.3f}".format(res_s.conf_int().values[1:, 0][0])+', '+"{:.3f}".format(res_s.conf_int().values[1:, 1][0])+'], p='+"{:.3e}".format(res_s.pvalues.values[1]))
ax[1].plot(plot_xs, res_d.params.values[0] + res_d.params.values[1]*plot_xd, linestyle='--', c='r', label='Slope='+"{:.3f}".format(res_d.params.values[1])+', CI = ['+"{:.3f}".format(res_d.conf_int().values[1:, 0][0])+', '+"{:.3f}".format(res_d.conf_int().values[1:, 1][0])+'], p='+"{:.3e}".format(res_d.pvalues.values[1]))
ax[0].legend()
ax[0].set_xticks([0,1,2,3,4])
ax[0].set_ylabel('∆SBP Distribution STD')
ax[0].set_xticklabels(['seconds','minutes','hours','days','months'])
ax[0].set_xlabel('Time between calibration and test')
ax[1].legend()
ax[1].set_ylabel('∆DBP Distribution STD')
ax[1].set_xlabel('Time between calibration and test')
ax[1].set_ylabel('∆SBP Distribution STD')
ax[1].set_xticklabels(['seconds','minutes','hours','days','months'])

In [ ]:
for i in df_search.index:
    dsearch = df_search.iloc[i]
    figname = 'supplementary/other_figs/' + dsearch.Reg2+'_'+dsearch.Reg1
    f = dsearch.Reg1+'~'+dsearch.Reg2
    if dsearch.Cat == None:
        plot_df = supp_df[[dsearch.Reg1, dsearch.Reg2]]
        sns.lmplot(data=plot_df, x=dsearch.Reg2, y=dsearch.Reg1, height=5)
    else:
        plot_df = supp_df[[dsearch.Reg1, dsearch.Reg2, dsearch.Cat]]
        f+='C('+dsearch.Cat+')'
        sns.lmplot(data=plot_df, x=dsearch.Reg2, y=dsearch.Reg1, hue=dsearch.Cat, height=5)
        figname+='_'+dsearch.Cat
    ax = plt.gca()
    plt.savefig(figname+'.png')
    plt.close()

In [ ]:
dsearch

# Create Benchmark Table

In [ ]:
benchmark_data = pd.read_excel('../metadata.xlsx', sheet_name='Benchmarks')
benchmark_key = pd.read_excel('../metadata.xlsx', sheet_name='Benchmark Key')
benchmark_data['ED'] = benchmark_data['SBP ED'].astype(str) + '/' + benchmark_data['DBP ED'].astype(str)

In [ ]:
def assign_name(d, key):
    if d == d:
        row = key[key['Benchmark Item Name'] == d]
        return row['Display Name'].values[0]
    else:
        pass

cols = ['Filter', 'Estimator']
for col in cols:
    benchmark_data[col] = benchmark_data[col].apply(assign_name, args=(benchmark_key, ))

In [ ]:
from bokeh.transform import factor_cmap, factor_mark
from bokeh.palettes import Category10_3
from bokeh.models import Legend, LegendItem

output_file('./_includes/replicated_ed_scatter.html')

hover = HoverTool(
        tooltips="""
        <div>
            <div>
                <span style="font-size: 10px"> Dataset: @{Dataset} </span> <br>
                <span style="font-size: 10px"> Filter: @{Filter} </span> <br>
                <span style="font-size: 10px"> Estimator: @{Estimator} </span> <br>
                <span style="font-size: 10px"> Reported SBP/DBP ED: @{Reported SBP ED} / @{Reported DBP ED}</span> <br>
                <span style="font-size: 10px"> Replicated SBP/DBP ED: @{SBP ED} / @{DBP ED}</span> <br>
            </div>
        </div>
        """
    )

p21 = figure(title = "Reported vs Replicated ED for SBP and DBP", background_fill_color="#fafafa", plot_width=400, plot_height=400, toolbar_location='above', tools=[hover, 'pan', 'wheel_zoom' ,'box_zoom', 'reset'])
p21.xaxis.axis_label = 'Reported SBP ED'
p21.yaxis.axis_label = 'Replicated SBP ED'
p22 = figure(title = "", background_fill_color="#fafafa", plot_width=485, plot_height=400, toolbar_location='above', tools=[hover, 'pan', 'wheel_zoom' ,'box_zoom', 'reset'])
p22.xaxis.axis_label = 'Reported DBP ED'
p22.yaxis.axis_label = 'Replicated DBP ED'

cs = ['#a6cee3', '#1f78b4', '#b2df8a', '#33a02c', '#fb9a99', '#e31a1c', '#fdbf6f', '#ff7f00', '#cab2d6', '#6a3d9a', '#ffff99', '#b15928', '#8dd3c7', '#ffffb3', '#bebada', '#fb8072', '#80b1d3', '#fdb462', '#b3de69', '#fccde5', '#d9d9d9', '#bc80bd', '#ccebc5', '#ffed6f']
shapes = ['circle', 'triangle', 'diamond', 'plus']
dset_unique = benchmark_data['Dataset'].unique()
filter_unique = benchmark_data['Filter'].unique()
color_dict = {}
for i in range(len(dset_unique)):
    color_dict[dset_unique[i]] = cs[i]
shape_dict = {}
for i in range(len(filter_unique)):
    shape_dict[filter_unique[i]] = shapes[i]
benchmark_data['color'] = benchmark_data['Dataset'].apply(lambda x: color_dict[x])
benchmark_data['shape'] = benchmark_data['Filter'].apply(lambda x: shape_dict[x])

legend_it = []
for i in range(len(dset_unique)):
    d = benchmark_data[benchmark_data['Dataset'] == dset_unique[i]]
    c1 = p21.scatter('Reported SBP ED', 'SBP ED', source=ColumnDataSource(d), size=12, color='color', marker='shape')
    c2 = p22.scatter('Reported DBP ED', 'DBP ED', source=ColumnDataSource(d), size=12, color='color', marker='shape')

    legend_it.append((dset_unique[i], [c1, c2]))
legend = Legend(items=legend_it)
legend.click_policy='hide'
p22.add_layout(legend, 'right')
p22.legend.title = 'Dataset'

show(row([p21, p22]))

In [ ]:
from bokeh.transform import factor_cmap, factor_mark
from bokeh.palettes import Category10_3
from bokeh.models import Legend, LegendItem

output_file('./_includes/replicated_ed_diff_scatter.html')

hover = HoverTool(
        tooltips="""
        <div>
            <div>
                <span style="font-size: 10px"> Dataset: @{Dataset} </span> <br>
                <span style="font-size: 10px"> Filter: @{Filter} </span> <br>
                <span style="font-size: 10px"> Estimator: @{Estimator} </span> <br>
                <span style="font-size: 10px"> Reported SBP/DBP ED: @{Reported SBP ED} / @{Reported DBP ED}</span> <br>
                <span style="font-size: 10px"> Replicated SBP/DBP ED: @{SBP ED} / @{DBP ED}</span> <br>
                <span style="font-size: 10px"> SBP/DBP ED Difference: @{SBP ED Difference} / @{DBP ED Difference} </span> <br>
            </div>
        </div>
        """
    )

p1 = figure(title = "Replicated-Reported ED Difference for SBP and DBP", background_fill_color="#fafafa", plot_width=550, plot_height=400, toolbar_location='above', tools=[hover, 'pan', 'wheel_zoom' ,'box_zoom', 'reset'])
p1.xaxis.axis_label = 'Replicated DBP ED - Reported DBP ED'
p1.yaxis.axis_label = 'Replicated SBP ED - Reported SBP ED'

cs = ['#a6cee3', '#1f78b4', '#b2df8a', '#33a02c', '#fb9a99', '#e31a1c', '#fdbf6f', '#ff7f00', '#cab2d6', '#6a3d9a', '#ffff99', '#b15928', '#8dd3c7', '#ffffb3', '#bebada', '#fb8072', '#80b1d3', '#fdb462', '#b3de69', '#fccde5', '#d9d9d9', '#bc80bd', '#ccebc5', '#ffed6f']
shapes = ['circle', 'triangle', 'diamond', 'plus']
dset_unique = benchmark_data['Dataset'].unique()
filter_unique = benchmark_data['Filter'].unique()
color_dict = {}
for i in range(len(dset_unique)):
    color_dict[dset_unique[i]] = cs[i]
shape_dict = {}
for i in range(len(filter_unique)):
    shape_dict[filter_unique[i]] = shapes[i]
benchmark_data['color'] = benchmark_data['Dataset'].apply(lambda x: color_dict[x])
benchmark_data['shape'] = benchmark_data['Filter'].apply(lambda x: shape_dict[x])
benchmark_data['SBP ED Difference'] = benchmark_data['SBP ED'].astype(float) - benchmark_data['Reported SBP ED'].astype(float)
benchmark_data['DBP ED Difference'] = benchmark_data['DBP ED'].astype(float) - benchmark_data['Reported DBP ED'].astype(float)

legend_it = []
for i in range(len(dset_unique)):
    d = benchmark_data[benchmark_data['Dataset'] == dset_unique[i]]
    c1 = p1.scatter('DBP ED Difference', 'SBP ED Difference', source=ColumnDataSource(d), size=12, color='color', marker='shape')
    legend_it.append((dset_unique[i], [c1]))
legend = Legend(items=legend_it)
legend.click_policy='hide'
p1.add_layout(legend, 'right')
p1.legend.title = 'Dataset'

show(p1)

In [ ]:
d = benchmark_data.copy().dropna(how='all')
def assign_url(d, key):
    if d == d:
        row = key[key['Display Name'] == d]
        return '<a href="' + row['URL'].values[0] + '">' + row['Display Name'].values[0] + '</a>'
    else:
        pass

cols = ['Filter', 'Estimator']
for col in cols:
    d[col] = d[col].apply(assign_url, args=(benchmark_key, ))

d = d.pivot(index=['Dataset', 'Filter'], columns='Estimator', values='ED').T

In [ ]:
d.reset_index().to_html('./_includes/benchmark_table.html', index=False, render_links=True, table_id='btable', escape=False)

# Create Dataset Table

In [ ]:
data = pd.read_excel('../metadata.xlsx', sheet_name='Publicly Available Datasets')

In [ ]:
data['Dataset Name'] = make_clickable(data['Dataset Name'], data['Dataset URL'])
data['Article URL'] = make_clickable('Article URL', data['Dataset URL'])

In [ ]:
data[['Dataset Name', 'Article URL', 'Sensor Data', 'Number of Subjects', 'Study Characteristics', 'BP Distribution SBP', 'BP Distribution DBP', 'Access', 'Notes']].to_html('./_includes/dataset_table.html', index=False, render_links=True, table_id='dtable', escape=False)

# Remove Doctype formatting

In [ ]:
import os
for fname in os.listdir('./_includes'):
    if ('scatter' in fname) | ('_stats' in fname):
        fpath = './_includes/' + fname
        with open(fpath, 'rb') as fin:
            data = fin.read().splitlines(True)
        with open(fpath, 'wb') as fout:
            fout.writelines(data[1:])